<a href="https://colab.research.google.com/github/Yashaswini1308/GenerativeAI_Mental-Heath-Support-Chatbot/blob/main/HealMindBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

%cd gdrive/My Drive

Mounted at /content/gdrive
/content/gdrive/My Drive


In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j tiktoken yfiles_jupyter_graphs streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 96.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.3

In [ ]:
pip install ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 25.8 MB/s eta 0:00:00


In [ ]:
pip install neo4j

In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
up to date, audited 23 packages in 5s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠋

In [ ]:
# %%writefile main.py
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough


try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass



/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
os.environ["OPENAI_API_KEY"] = ""#replace your OPEN_API key
os.environ["NEO4J_URI"] = "neo4j+s://c1cdf9bd.databases.neo4j.io"#NEO4J URL
os.environ["NEO4J_USERNAME"] = "neo4j" #NEO4J USERNAME
os.environ["NEO4J_PASSWORD"] = "NGp9xNPgGPdFcAQt8SvL2Nzl_s3HeIzfksIXq_HpGMQ"#NEO4J PASSWORD

In [ ]:
graph = Neo4jGraph()

<ipython-input-9-6e7385464d00>:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [ ]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 16.1 MB/s eta 0:00:00


In [ ]:
from langchain_community.document_loaders import JSONLoader

# Path to your local JSON file
file_path = "/content/gdrive/MyDrive/intents.json"

# Create the JSON loader
loader = JSONLoader(
    file_path=file_path,
    jq_schema=".intents[]",  # Extract each intent object
    text_content=False       # We'll structure the text ourselves later
)

# Load the data
docs = loader.load()

In [ ]:
for doc in docs[:500]:  # Show just a few examples
    print(doc.page_content)

{"tag": "greeting", "patterns": ["Hi", "Hey", "Is anyone there?", "Hi there", "Hello", "Hey there", "Howdy", "Hola", "Bonjour", "Konnichiwa", "Guten tag", "Ola"], "responses": ["Hello there. Tell me how are you feeling today?", "Hi there. What brings you here today?", "Hi there. How are you feeling today?", "Great to see you. How do you feel currently?", "Hello there. Glad to see you're back. What's going on in your world right now?"]}
{"tag": "morning", "patterns": ["Good morning"], "responses": ["Good morning. I hope you had a good night's sleep. How are you feeling today? "]}
{"tag": "afternoon", "patterns": ["Good afternoon"], "responses": ["Good afternoon. How is your day going?"]}
{"tag": "evening", "patterns": ["Good evening"], "responses": ["Good evening. How has your day been?"]}
{"tag": "night", "patterns": ["Good night"], "responses": ["Good night. Get some proper sleep", "Good night. Sweet dreams."]}
{"tag": "goodbye", "patterns": ["Bye", "See you later", "Goodbye", "Au rev

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(docs)

In [ ]:
print(documents[0])
print(documents[1])
print(documents[2])

page_content='{"tag": "greeting", "patterns": ["Hi", "Hey", "Is anyone there?", "Hi there", "Hello", "Hey there", "Howdy", "Hola", "Bonjour", "Konnichiwa", "Guten tag", "Ola"], "responses": ["Hello there. Tell me how are you feeling today?", "Hi there. What brings you here today?", "Hi there. How are you feeling today?", "Great to see you. How do you feel currently?", "Hello there. Glad to see you're back. What's going on in your world right now?"]}' metadata={'source': '/content/gdrive/MyDrive/intents.json', 'seq_num': 1}
page_content='{"tag": "morning", "patterns": ["Good morning"], "responses": ["Good morning. I hope you had a good night's sleep. How are you feeling today? "]}' metadata={'source': '/content/gdrive/MyDrive/intents.json', 'seq_num': 2}
page_content='{"tag": "afternoon", "patterns": ["Good afternoon"], "responses": ["Good afternoon. How is your day going?"]}' metadata={'source': '/content/gdrive/MyDrive/intents.json', 'seq_num': 3}


In [ ]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125") # gpt-4-0125-preview occasionally has issues
llm_transformer = LLMGraphTransformer(llm=llm)

# Following needs to be used only first time to load the data. It took 3m 44s
graph_documents = llm_transformer.convert_to_graph_documents(documents)


/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1673: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
print (documents[5])
print(graph_documents[5])

page_content='{"tag": "goodbye", "patterns": ["Bye", "See you later", "Goodbye", "Au revoir", "Sayonara", "ok bye", "Bye then", "Fare thee well"], "responses": ["See you later.", "Have a nice day.", "Bye! Come back again.", "I'll see you soon."]}' metadata={'source': '/content/gdrive/MyDrive/intents.json', 'seq_num': 6, 'id': '4188fce5648ddcb5bfe61937b668ecbb'}
nodes=[] relationships=[] source=Document(metadata={'source': '/content/gdrive/MyDrive/intents.json', 'seq_num': 6, 'id': '4188fce5648ddcb5bfe61937b668ecbb'}, page_content='{"tag": "goodbye", "patterns": ["Bye", "See you later", "Goodbye", "Au revoir", "Sayonara", "ok bye", "Bye then", "Fare thee well"], "responses": ["See you later.", "Have a nice day.", "Bye! Come back again.", "I\'ll see you soon."]}')


In [ ]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
# Create index for intent-related entities
graph.query(
    "CREATE FULLTEXT INDEX intent IF NOT EXISTS FOR (i:__Intent__) ON EACH [i.tag]"
)

# New Pydantic model for intent structure
class IntentInfo(BaseModel):
    """Identifying information about intents."""

    tag: str = Field(..., description="The label representing the intent.")
    patterns: List[str] = Field(..., description="Common user inputs for this intent.")
    responses: List[str] = Field(..., description="Bot responses corresponding to this intent.")

# Updated prompt template for intent extraction
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting intents from the text, including tag, patterns, and responses.",
        ),
        (
            "human",
            "Extract intents from the following input using the format with fields: tag, patterns, responses:\n\n{question}",
        ),
    ]
)

# Updated entity_chain (now for intent extraction)
intent_chain = prompt | llm.with_structured_output(IntentInfo)


/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1660: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1673: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo-0125 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [ ]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query from the input, allowing fuzzy matches.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Updated for intent-based retrieval
def structured_intent_retriever(question: str) -> str:
    """
    Collects neighborhood of intents relevant to the user's question.
    """
    result = ""
    intent_info = intent_chain.invoke({"question": question})

    # Use the extracted 'tag' from the intent to query the graph
    tag = intent_info.tag
    query_string = generate_full_text_query(tag)

    response = graph.query(
        """
        CALL db.index.fulltext.queryNodes('intent', $query, {limit: 2})
        YIELD node, score
        CALL {
          WITH node
          MATCH (node)-[r:!MENTIONS]->(neighbor)
          RETURN node.tag + ' - ' + type(r) + ' -> ' + neighbor.tag AS output
          UNION ALL
          WITH node
          MATCH (node)<-[r:!MENTIONS]-(neighbor)
          RETURN neighbor.tag + ' - ' + type(r) + ' -> ' +  node.tag AS output
        }
        RETURN output LIMIT 50
        """,
        {"query": query_string},
    )
    result += "\n".join([el['output'] for el in response])
    return result


In [ ]:
def retriever(question: str):
    print(f"Search query: {question}")

    structured_data = structured_intent_retriever(question)
    unstructured_docs = [el.page_content for el in vector_index.similarity_search(question)]

    # Perform the join before using it inside f-string
    joined_unstructured = "\n# Document ".join(unstructured_docs)

    final_data = f"""Structured data:
{structured_data}

Unstructured data:
# Document {joined_unstructured}
"""
    return final_data



In [ ]:
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, BaseMessage
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnableLambda, RunnableBranch, RunnablePassthrough

# Prompt
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

# Format chat history into proper message objects
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List[BaseMessage]:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

# Main branching logic
_search_query = RunnableBranch(
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(run_name="HasChatHistoryCheck"),
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser()
    ),
    RunnableLambda(lambda x: x["question"])  # If no chat history
)


<ipython-input-27-5aee3dfab922>:32: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  | ChatOpenAI(temperature=0)


In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chat_models import ChatOpenAI

# Prompt
template = """Answer the query:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# LLM setup
llm = ChatOpenAI(temperature=0)

# Full QA chain
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,  # standalone question → retriever
            "question": RunnablePassthrough(),     # pass through original question
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
chain.invoke({"question": "I am depressed"})

Search query: I am depressed


<ipython-input-25-0f71018f7fa3>:6: LangChainDeprecationWarning: The function `remove_lucene_chars` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the function exists in the :meth:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :meth:`~langchain-neo4j` and import as `from :meth:`~langchain_neo4j.vectorstores.neo4j_vector import remove_lucene_chars``.
  words = [el for el in remove_lucene_chars(input).split() if el]


"It helps to talk about what's happening. You're going to be okay. Talk to me and tell me more. It helps if you open up yourself to someone else. Sometimes when we are depressed, it is hard to care about anything. Give yourself time to heal."

In [ ]:
chain.invoke({"question": "I don't need your help at all"})

Search query: I don't need your help at all


"That's okay, I'm here if you change your mind."

In [ ]:
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger("neo4j.notifications").setLevel(logging.ERROR)

In [ ]:
chain.invoke({"question": "What's the difference between anxiety and stress?"})

Search query: What's the difference between anxiety and stress?


'Anxiety and stress are often used interchangeably, but they have different causes. Stress is related to external pressures we find hard to cope with, while anxiety focuses on worries or fears about potential threats. Stress symptoms typically disappear after the stressful situation ends, while anxiety can be harder to pinpoint and may persist.'

In [ ]:
chain.invoke({"question": "I am getting suicidal thoughts"})

Search query: I am getting suicidal thoughts


"I'm very sorry to hear that you are experiencing suicidal thoughts. Please seek help by contacting: 9152987821. It's important to talk to someone you trust and get the support you need."

In [ ]:
chain.invoke({"question": "You are not helping me , I don't like you anymore"})

Search query: You are not helping me , I don't like you anymore


"I'm sorry to hear that you feel that way. I'm here to help you, so please feel free to share your thoughts and concerns with me."

In [ ]:
chain.invoke({"question": "Is fire hot tell me T or F?"})

Search query: Is fire hot tell me T or F?


'True, fire is hot.'

In [ ]:
!streamlit run app.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.118.161:8501

